# LABORATORIO #1 
 Autores: Alfredo Quezada
 
 Autores: Randy Venegas

### Parte 1

* 1. Cargue el dataset en un dataframe de pandas, muestre un ejemplo de cinco observaciones. 

In [15]:
import pandas as pd
from urllib.parse import urlparse

df = pd.read_csv('dataset_pishing.csv')
df.sample(n=5).head()

,url,ip,nb_www,nb_com,nb_dslash,http_in_path,punycode,port,tld_in_path,tld_in_subdomain,...,domain_in_title,domain_with_copyright,whois_registered_domain,domain_registration_length,domain_age,web_traffic,dns_record,google_index,page_rank,status
8931,http://mail.schoonheidssalonlafinesse.nl/~u437...,0,0,0,0,0,0,0,0,0,...,1,0,0,0,936,0,0,1,0,phishing
5608,http://houseoftiresbcs.com/Adobe/css/XML/PDF/g...,0,0,0,0,0,0,0,0,0,...,1,0,0,53,1407,0,0,1,0,phishing
6956,http://disney.wikia.com/wiki/The_Mickey_Mouse_...,0,0,0,0,0,0,0,0,0,...,1,1,0,140,6070,14592,0,0,5,legitimate
2171,https://en.wikipedia.org/wiki/Computer_mouse,0,0,1,0,0,0,0,0,0,...,0,1,0,902,7133,12,0,0,7,legitimate
11110,https://shop.hostingdude.com/?q=http://take-aw...,0,0,1,1,1,0,0,1,0,...,1,1,0,3491,6735,512410,0,0,2,phishing


* 2. Muestre la cantidad de observaciones etiquetadas en la columna status como “legit” y como 
“pishing”. ¿Está balanceado el dataset? 

In [16]:
print(df['status'].value_counts())

legitimate    5715
phishing      5715
Name: status, dtype: int64


        El modelo, como se puede observar si se encuentra balanceado. 

* 3. Para ello escriba las funciones necesarias y genere las nuevas columnas del dataset. Muestre un nuevo 
ejemplo de cinco observaciones donde se visualicen algunas de las columnas nuevas. 

In [17]:
#In the first function, the goal is to find the length of the domain of the page
import re

def domain_length(url):
    parsed_uri = urlparse(url)
    domain_name = '{uri.scheme}://{uri.netloc}/'.format(uri=parsed_uri)
    return len(domain_name)


df_with_features = df.assign(domain_length = df['url'].apply(domain_length))



#In the second function, special characters are counted
def count_special_characters(url):
    if re.search(r'[.,@/?&|=~#%$:*;,! ]', url):
        return 0
    else:
        return 1

df_with_features['has_special_characters'] = df_with_features['url'].apply(count_special_characters)


#The third function to see the digits in the domain names
def count_digits_in_domain(url):
    return len(re.findall(r'\d', urlparse(url).netloc))
df_with_features['digits_in_domain'] = df_with_features['url'].apply(count_digits_in_domain)

#The fourth function is to have the https in the url
def has_https(url):
    if 'https' in url:
        return 1
    else:
        return 0
df_with_features['has_https'] = df_with_features['url'].apply(has_https)

#The fifth function is to obtain the digits in the url
def count_digits(url):
    return len(re.findall(r'\d', url))
df_with_features['digits_count'] = df_with_features['url'].apply(count_digits)

df_with_features.sample(n=5).head()



,url,ip,nb_www,nb_com,nb_dslash,http_in_path,punycode,port,tld_in_path,tld_in_subdomain,...,web_traffic,dns_record,google_index,page_rank,status,domain_length,has_special_characters,digits_in_domain,has_https,digits_count
1545,http://www.kidscoloringpdf.com/wp-content/xxx/...,0,1,0,0,0,0,0,0,0,...,0,0,1,0,phishing,31,0,0,0,4
5846,https://www.matchy.net/,0,1,0,0,0,0,0,0,0,...,0,0,0,4,legitimate,23,0,0,1,0
3804,https://startxpo.com/facture_orange/customer_c...,0,0,0,0,0,0,0,0,0,...,0,0,1,0,phishing,21,0,0,1,5
4262,http://ecopointeu.com/oyi/Exitkorea/upload/ind...,0,0,1,0,0,0,0,1,0,...,10469481,0,1,0,phishing,22,0,0,0,0
1012,http://true654263721ffuyfsfs.clearpointsupplie...,1,0,0,0,0,0,0,0,0,...,0,0,1,0,phishing,52,0,9,0,9


Cambiar la variable a binaria 

In [18]:
df_with_features['status'] = df_with_features['status'].map({'legitimate': 1, 'phishing': 0})
df_with_features = df_with_features.drop('url', axis=1)
df_with_features.head()

,ip,nb_www,nb_com,nb_dslash,http_in_path,punycode,port,tld_in_path,tld_in_subdomain,abnormal_subdomain,...,web_traffic,dns_record,google_index,page_rank,status,domain_length,has_special_characters,digits_in_domain,has_https,digits_count
0,0,1,0,0,0,0,0,0,0,0,...,0,1,1,4,1,27,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,1,2,0,31,0,0,0,17
2,1,0,1,0,0,0,0,0,1,0,...,5828815,0,1,0,0,59,0,0,1,19
3,0,0,0,0,0,0,0,0,0,0,...,107721,0,0,3,1,19,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,8725,0,0,6,1,23,0,0,0,0


Visualizacion


In [20]:
import pandas_profiling


In [21]:
# save the report to a html file
pandas_profiling.ProfileReport(df_with_features).to_file('report.html')


TypeError: ignored